# Developing NanoGPT

## Setup

In [25]:
import torch
torch.manual_seed(1337)

In [2]:
"""Download Shakespeare training dataset"""
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-10-01 12:23:12--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2023-10-01 12:23:13 (8.87 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [5]:
"""Read Shakespeare file"""
with open('input.txt', 'r', encoding='utf-8') as file:
	text = file.read()

In [8]:
print(f'The Shakespeare file has {len(text):,} characters')

The Shakespeare file has 1,115,394 characters


In [9]:
"""Print first 1K characters"""
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [10]:
unique_chars = sorted(list(set(text)))
vocab_size = len(unique_chars)
print(f'The Shakespeare file has {vocab_size} unique characters')
str_chars = ''.join(unique_chars)
print(f'Vocabulary: {str_chars}')


The Shakespeare file has 65 unique characters
Vocabulary: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


## Tokenization

In [15]:
"""Map characters to integers"""
string_to_int = {char: idx for idx, char in enumerate(unique_chars)}
int_to_string = {idx: char for idx, char in enumerate(unique_chars)}

In [12]:
"""Takes a string as input and outputs a list of integers"""
encode = lambda string: [string_to_int[char] for char in string]
"""Takes a list of integers as input and outputs string"""
decode = lambda integers: ''.join([int_to_string[idx] for idx in integers])

In [16]:
print(encode('test'))
print(decode(encode('test')))

[58, 43, 57, 58]
test


## Data Storage

In [18]:
"""Encode entire text and store in a toch tensor"""
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


/var/folders/v3/_27cdtx91k7c6brtzn096lx00000gq/T/ipykernel_99248/1838959051.py:2: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  data = torch.tensor(encode(text), dtype=torch.long)


In [19]:
print(data[:100])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [20]:
"""Split data into training (90%) and validation (10%)"""
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

## Chunking
- Split training data into sampled chunks with a block size (token length) of 8
- For a block size of 8, we actually need 9 chars because the sample is the next predicted char, and we need some context for the next prediction. So, to predict 8 times, we need 9 chars

In [23]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [22]:
"""
Here, X are the input characters and y are the target characters
Again, we need to offset by 1 for the targets so that the model has some context to predict from
"""
x = train_data[:block_size]
y = train_data[1:block_size + 1]

In [24]:
for t in range(block_size):
	"""Context is always the characters in x up to and including t"""
	context = x[:t + 1]
	target = y[t]
	print(f'When input is {context}, target is {target}')

When input is tensor([18]), target is 47
When input is tensor([18, 47]), target is 56
When input is tensor([18, 47, 56]), target is 57
When input is tensor([18, 47, 56, 57]), target is 58
When input is tensor([18, 47, 56, 57, 58]), target is 1
When input is tensor([18, 47, 56, 57, 58,  1]), target is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]), target is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), target is 58


## Batching and Blocking
- Batch size is the number of independent sequences we want to process in parallel on every forward-backward pass of the transformer
- Block size is the maximum content length in tokens used to make predictions

In [26]:
batch_size = 4
block_size = 8

In [28]:
def get_batch(split:str):
	"""The function generates a small batch of data: inputs x and targets y"""
	data = train_data if split == 'train' else val_data
	
	"""Generate 4 random numbers between 0 and n - block_size
	These will be the starting indices of the 4 sequences in the batch"""
	n = len(data)
	rand_starting_points = torch.randint(n - block_size, (batch_size,))
	"""
	Stack up the inputs and targets for each sequence in the batch into a 4 x 8 tensor
	The target sequence for each input sequence is the same as the input sequence, but shifted by one character to the right
	"""
	x = torch.stack(data[point:point + block_size] for point in rand_starting_points)
	y = torch.stack([data[point + 1:point + block_size + 1] for point in rand_starting_points])
	return x, y